# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [48]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [49]:
dropdown = Select(driver.find_element_by_name('pht_status'))
dropdown.select_by_visible_text("Cosmetologists")

In [50]:
driver.find_element_by_name('pht_lnm').send_keys('nguyen')

In [51]:
button = driver.find_element_by_name('B1')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [31]:
results = driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    print(result.text)

Name and Location Order Basis for Order
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Responde

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [34]:
results = driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    try:
        name = result.find_element_by_class_name('results_text')
        print(name.text)
    except:
        print("Doesn't have a name")

Doesn't have a name
NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [37]:
results = driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    try:
        cells = result.find_elements_by_tag_name('td')
        print(cells[2].text)
    except:
        print("Doesn't have a violation")

Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinf

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [52]:
results = driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    try:
        cells = result.find_elements_by_class_name('results_text')
        print(cells[-2].text)
    except:
        print("Doesn't have a complaint number")

Doesn't have a complaint number
COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [58]:
rows = []

results = driver.find_elements_by_tag_name('tr')
for result in results:
    try:
        row = {}
        cells = result.find_elements_by_tag_name('td')
        results = result.find_elements_by_class_name('results_text')
        row['name'] = results[0].text
        row['city'] = results[1].text
        row['county'] = results[2].text
        row['zip_code'] = results[3].text
        row['complaint_no'] = results[-2].text
        row['license_numbers'] = results[-3].text
        row['complaint'] = cells[-1].text
        rows.append(row)
    except:
        print("This row is broken:", result.text)

Name and Location Order Basis for Order


### Save that to a CSV

- Tip: You'll want to use pandas here

In [47]:
import pandas as pd

df = pd.DataFrame(rows)
df.head()

,city,complaint,complaint_no,county,license_numbers,name,zip_code
0,SAN ANTONIO,Respondent failed to clean and sanitize whirlp...,COS20180004289,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",78217
1,EL PASO,Respondent failed to clean and sanitize whirlp...,COS20180006594,EL PASO,737708,"NGUYEN, HANH CONG",79934
2,LONGVIEW,Respondent failed to follow whirlpool foot spa...,COS20180000257,GREGG,731665,"NGUYEN, KHIEM VAN",75604
3,HOUSTON,"Respondent failed to disinfect tools, implemen...",COS20180004915,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC",77014
4,SAN ANTONIO,"Respondent failed to clean, disinfect, and ste...",COS20180009255,BEXAR,767339,"NGUYEN, LAN T-THUY",78255


In [59]:
df.to_csv("output.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.